# Transformers Tesis

Continuación del desarrollo de la tesis, solo con Transformers y los archivos generados anteriormente.

## Install

In [26]:
!pip install numpy h5py torch


886.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [27]:
# import packages
import os
import numpy as np
import h5py
# from drive.MyDrive.TESIS.TESIS_FINAL.Desarrollo.bmi.utils import data_loader
# from scipy import stats
import platform
# import sklearn
import torch

print(f"Python (v{platform.python_version()})")
# print(f"Sklearn (v{sklearn.__version__})")
print(f"Pytorch (v{torch.__version__})")

Python (v3.8.10)
Pytorch (v2.0.1+cu117)


## Lectura dataset para un archivo:

drive > MyDrive > TESIS > TESIS_FINAL > Desarrollo > data > dataset > indy_20160407_02_baks.h5

In [28]:
# archivo más pequeño 4,8 
filepath_dataset = 'indy_20161005_06_baks.h5'
filepath_dataset

'indy_20161005_06_baks.h5'

In [29]:
import torch

class Dataset_tesis(torch.utils.data.Dataset):
    def __init__(self, filepath_dataset: str, feature: str, velocity: bool =True):
        """
        Lee el archivo dataset para trabajar con DL.
        ------------
        Parámetros: 
        filepath_dataset: String
            Dirección donde se enceuntra el dataset.
        feature: np array
            SUA o MUA.
        velocity: Boolean
            Si quiere solo la velocidad para la salida.
        -------------
        Retorna:
        X: np array
            Dataset SUA o MUA, tasa de spikes estimada.
        y: np array
            Si velocity=True, entonces y solo contiene velocidad x e y del mono, si velocity=False, entonces y tiene posición, velocidad y aceleración x e y del mono.
        """
        with h5py.File(filepath_dataset, 'r') as f:
            self.X = f[f'X_{feature}'][()]
            self.y = f['y_task'][()]   
        if velocity:
            # select the x-y velocity components
            self.y = self.y[:,2:4] # data shape: n x 6 (x-y position, x-y velocity, x-y acceleration)

        assert len(self.X) == len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

In [30]:
# prueba
ds = Dataset_tesis(filepath_dataset, "sua")
display(ds[0])


(array([ 9.20983517,  5.24062105,  0.        ,  0.        ,  0.        ,
         2.03734188,  0.76785673,  2.00309387,  5.19855254,  0.7720459 ,
         7.18813229,  0.        ,  0.        ,  9.18557871,  0.        ,
         2.00916556,  0.7717755 ,  7.07515127,  3.52485634,  0.77222477,
         0.        ,  5.27834563,  2.02913591,  0.        ,  0.76401624,
         3.52105503,  2.01591345,  0.77329827,  7.07939913,  0.76122755,
         0.        ,  0.76925464,  2.03552179,  0.76916057,  0.7504905 ,
         5.2603572 ,  0.        ,  7.01707473,  0.        ,  0.        ,
         0.        ,  3.54606191,  3.46932884,  0.        ,  0.75119236,
         0.77326562,  0.77082073,  0.        ,  0.7726312 ,  0.75508565,
         0.        ,  0.76629387,  0.        ,  0.75198997,  0.        ,
        11.49041909,  0.        ,  0.        ,  0.        ,  3.48744758,
         3.45897978,  3.55597981,  0.76249352,  2.03660978,  3.53303551,
         0.        ,  0.        ,  0.76948741,  0. 

### Iterar sobre los datos

Dado un Dataset, sea hecho por nosotros o uno predefinido, iterar sobre los datos es muy sencillo. Simplemente tenemos que crear un objeto DataLoader que toma como argumento el Dataset y definir el batch size con que queremos trabajar.


Este código iterará por todos los batches de ejemplos de nuestro dataset y parará cuando se acaben. Es decir, esto corre por una **época**.

In [31]:
from torch.utils.data import DataLoader
train_dl = DataLoader(ds, batch_size=32)

for i, (x, target) in enumerate(train_dl):
    print(x.shape, target)
    if i==2:
      break

torch.Size([32, 116]) tensor([[-1.5632e+00,  4.1326e+00],
        [-4.9747e-01,  2.9586e+00],
        [-2.9077e-01,  1.5623e+00],
        [-2.8652e+00,  2.9060e+00],
        [-1.2743e+01,  1.2830e+01],
        [ 1.6147e+00, -1.1295e+00],
        [-1.6560e+01,  1.4538e+01],
        [-1.4289e+01,  1.4912e+01],
        [-3.3338e+01,  4.4084e+01],
        [-5.2631e+01,  1.3322e+02],
        [-8.4908e+00,  2.5790e+02],
        [-1.2324e+02,  1.3903e+02],
        [-2.0541e+01, -2.9118e+00],
        [-1.1909e+01, -3.2436e+00],
        [-1.6900e+01,  2.4566e+00],
        [-2.1415e+01, -5.0318e+01],
        [-1.4349e+00,  6.7594e+00],
        [ 2.9231e-01,  1.7495e+00],
        [-6.6488e+00,  4.2459e+00],
        [ 3.7126e+00, -2.1089e+00],
        [ 1.5310e+02, -1.0968e+02],
        [ 3.0639e+02, -2.1895e+02],
        [ 1.4029e+02, -3.0010e+01],
        [-4.3187e+01,  1.0594e+01],
        [ 3.2984e+01, -3.2205e+01],
        [-1.9657e+00,  2.0390e+00],
        [ 2.7659e-01, -2.9754e-01],
      

## Time2Vector

In [23]:
from time2vec.periodic_activations import SineActivation, CosineActivation
import torch
from torch.utils.data import DataLoader
from time2vec.Pipeline import AbstractPipelineClass
from torch import nn
from time2vec.Model import Model

class Tesis_t2v(AbstractPipelineClass):
    def __init__(self, model, ds):
        self.model = model
        self.ds = ds
    
    def train(self):
        loss_fn = nn.CrossEntropyLoss()

        train_dl = DataLoader(self.ds, batch_size=32, shuffle=False)

        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

        num_epochs = 2

        for ep in range(num_epochs):
            for x, y in train_dl:
                optimizer.zero_grad()

                y_pred = self.model(x.unsqueeze(1))
                loss = loss_fn(y_pred, y)

                loss.backward()
                optimizer.step()
                
                print(f"epoch: {ep}, loss:{loss.item()}")
    
    def preprocess(self, x):
        return x
    
    def decorate_output(self, x):
        return x


In [32]:
pipe = Tesis_t2v(Model("sin", 117), ds)
pipe.train()

#pipe = ToyPipeline(Model("cos", 12))
#pipe.train()

NameError: name 't1' is not defined

In [25]:
for x, y in train_dl:
  print(x.unsqueeze(1).float())
  break

tensor([[[ 9.2098,  5.2406,  0.0000,  ..., 11.1107, 15.5630,  0.0000]],

        [[ 5.1875,  3.5582,  0.0000,  ..., 23.2573, 13.6532,  0.0000]],

        [[ 2.0124,  5.1162,  0.7699,  ..., 20.7164, 11.1603,  0.0000]],

        ...,

        [[ 2.0202,  0.7734,  0.0000,  ...,  2.0309, 13.7219,  0.0000]],

        [[ 3.4919,  0.0000,  1.9400,  ...,  1.9988,  6.9782,  0.0000]],

        [[ 0.7712,  0.7620,  2.0408,  ...,  3.4933,  7.2585,  0.0000]]])
